In [1]:
#importando bibliotecas
from random import randrange
import os.path
import requests
import json
import os
import configparser

# Nova seção

Crie Um arquivo 'config.ini' e cole o seguinte texto:

[API]
key = insira_a_chave_aqui

Substitua a chave da API na variável key, mais informacoes sobre a chave API: https://platform.openai.com/account/api-keys


In [3]:
#Carregando Arquivos de Configuração
config = configparser.ConfigParser()
config.read('config.ini')
API_KEY = config['API']['key']
#openai.api_key = API_KEY
arquivo_LATEX = ''''''
dicionarios_questoes = []
#MODELO gpt-3.5-turbo	atual
modelo_3_5_atual = "gpt-3.5-turbo"
#MODELO gpt-3.5-turbo	versao antiga
modelo_3_5_0613   = "gpt-3.5-turbo-0613"
modelo_4o_08_out_24 = "gpt-4o-mini"
modelo_4oGRANDE_08_out_24 = "gpt-4o"
modelo_preview = "o1-preview"


In [4]:
questoes_TA = dict()
questoes_TA['Q1'] = r'$\vdash (A\lor (A\land B))\rightarrow A$'
questoes_TA['Q2'] = r'$\vdash (A\land (A\lor B))\rightarrow A$'
questoes_TA['Q3'] = r'$A\rightarrow (B\rightarrow C) \vdash B\rightarrow (A\rightarrow C)$'
questoes_TA['Q4'] = r'$\vdash (A\rightarrow (A\rightarrow B))\rightarrow (A\rightarrow B)$'
questoes_TA['Q5'] = r'$\vdash (\lnot A\rightarrow B)\rightarrow ((\lnot A\rightarrow \lnot B)\rightarrow A)$'
questoes_TA['Q6'] = r'$\vdash A\lor \lnot A$'
questoes_TA['Q7'] = r'$\vdash (A\rightarrow B)\lor (B\rightarrow A)$'
questoes_TA['Q8'] = r'$\vdash A\rightarrow A$'
questoes_TA['Q9'] = r'$\vdash (A\rightarrow B)\rightarrow ((C\rightarrow A)\rightarrow (C\rightarrow A))$'
questoes_TA['Q10'] = r'$(A\land B)\rightarrow C \vdash B\rightarrow (A\rightarrow C)$'
questoes_TA['Q11'] = r'$B\rightarrow (A\rightarrow C) \vdash (A\land B)\rightarrow C$'
questoes_TA['Q12'] = r'$A\rightarrow (B\rightarrow C) \vdash (A\rightarrow B)\rightarrow (A\rightarrow C)$'
questoes_TA['Q13'] = r'$\vdash A\rightarrow (B\rightarrow A)$'
questoes_TA['Q14'] = r'$\vdash ((A\rightarrow B)\rightarrow A)\rightarrow A$'
questoes_TA['Q15'] = r'$A\lor B, A\rightarrow C, B\rightarrow C \vdash C$'
questoes_TA['Q16'] = r'$A \vdash \lnot \lnot A$'
questoes_TA['Q17'] = r'$\lnot \lnot A \vdash A$'
questoes_TA['Q18'] = r'$A\rightarrow B, \lnot B \vdash \lnot A$'
questoes_TA['Q19'] = r'$\lnot B\rightarrow \lnot A \vdash A\rightarrow B$'
questoes_TA['Q20'] = r'$A\rightarrow B \vdash \lnot B\rightarrow \lnot A$'
questoes_TA['Q21'] = r'$\lnot(A\lor B) \vdash \lnot A\land \lnot B$'
questoes_TA['Q22'] = r'$\lnot A\land \lnot B \vdash \lnot(A\lor B)$'
questoes_TA['Q23'] = r'$\lnot(A\land B) \vdash \lnot A\lor \lnot B$'
questoes_TA['Q24'] = r'$\lnot A\lor \lnot B \vdash \lnot(A\land B)$'
questoes_TA['Q25'] = r'$\lnot A\lor B \vdash A\rightarrow B$'
questoes_TA['Q26'] = r'$(A\lor B)\land (A\lor C) \vdash A\lor (B\land C)$'
questoes_TA['Q27'] = r'$A\land (B\lor C) \vdash (A\land B)\lor (A\land C)$'
questoes_TA['Q28'] = r'$(A\land B)\lor (A\land C) \vdash A\land (B\lor C)$'
questoes_TA['Q29'] = r'$A\lor B, \lnot B \vdash A$'
questoes_TA['Q30'] = r'$A\lor B \vdash \lnot A\rightarrow B$'
questoes_TA['Q31'] = r'$\lnot A\rightarrow B \vdash A\lor B$'
questoes_TA['Q32'] = r'$A\land B \vdash \lnot(A\rightarrow \lnot B)$'
questoes_TA['Q33'] = r'$\lnot(A\rightarrow \lnot B) \vdash A\land B$'
questoes_TA['Q34'] = r'$A\lor B \vdash \lnot(\lnot A\land \lnot B)$'
questoes_TA['Q35'] = r'$\lnot(\lnot A\land \lnot B) \vdash A\lor B$'
questoes_TA['Q36'] = r'$A\rightarrow B \vdash \lnot(A\land \lnot B)$'
questoes_TA['Q37'] = r'$\lnot(A\land \lnot B) \vdash A\rightarrow B$'
questoes_TA['Q38'] = r'$A\land B \vdash \lnot(\lnot A\lor \lnot B)$'
questoes_TA['Q39'] = r'$\lnot(\lnot A\lor \lnot B) \vdash A\land B$'
questoes_TA['Q40'] = r'$A\rightarrow B \vdash \lnot A\lor B$'


In [5]:
# A funcao recebe a chave da api e a conversa de entrada
# retorna a resposda da API
def chamada_api_chatGPT(API_KEY,conversa_entrada):
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type":"application/json"}
    link = "https://api.openai.com/v1/chat/completions"

    id_modelo = modelo_4oGRANDE_08_out_24
    body_mensagem = {
        "model": id_modelo,
        #"temperature" : 0.2,
        "messages": conversa_entrada
    }
    body_mensagem = json.dumps(body_mensagem)
    resposta = requests.post(link,headers=headers,data=body_mensagem)
    resposta = resposta.json()
    return resposta

In [6]:
# Texto Padrao da primeira mensagem do usuario
user_content = r"""escreva a prova $A\lor (B\land C) \vdash (A\lor B)\land (A\lor C)$ usando o sistema de tableaux analitico na linguagem latex na biblioteca qtree"""
# Texto Padrao da primeira mensagem do assistente
assistant_response = r"""
\Tree [.{$T~A\lor (B\land C)$ \\ $F~(A\lor B)\land (A\lor C)$} [.{$F~A\lor B$} [.{{\color{blue}$F~A$} \\ {\color{blue}$F~B$}} [.{\color{blue}{$T~A$}} [.{$\times$} ] ] [.{$T~B\land C$} [.{{\color{blue}$T~B$} \\ $T~C$} [.{$\times$} ] ] ] ] ] [.{$F~A\lor C$} [.{{\color{blue}$F~A$} \\ {\color{blue}$F~C$}} [.{\color{blue}{$T~A$}} [.{$\times$} ] ] [.{$T~B\land C$} [.{$T~B$ \\ {\color{blue}$T~C$}} [.{$\times$} ] ] ] ] ] ]
"""
print(user_content)
print(assistant_response)

escreva a prova $A\lor (B\land C) \vdash (A\lor B)\land (A\lor C)$ usando o sistema de tableaux analitico na linguagem latex na biblioteca qtree

\Tree [.{$T~A\lor (B\land C)$ \\ $F~(A\lor B)\land (A\lor C)$} [.{$F~A\lor B$} [.{{\color{blue}$F~A$} \\ {\color{blue}$F~B$}} [.{\color{blue}{$T~A$}} [.{$\times$} ] ] [.{$T~B\land C$} [.{{\color{blue}$T~B$} \\ $T~C$} [.{$\times$} ] ] ] ] ] [.{$F~A\lor C$} [.{{\color{blue}$F~A$} \\ {\color{blue}$F~C$}} [.{\color{blue}{$T~A$}} [.{$\times$} ] ] [.{$T~B\land C$} [.{$T~B$ \\ {\color{blue}$T~C$}} [.{$\times$} ] ] ] ] ] ]



In [7]:
user_question = f"""escreva a prova ${questoes_TA['Q1']}$ usando o sistema de tableaux analitico na linguagem latex na biblioteca qtree"""
user_question

'escreva a prova $$\\vdash (A\\lor (A\\land B))\\rightarrow A$$ usando o sistema de tableaux analitico na linguagem latex na biblioteca qtree'

In [15]:
for x in range(29,31):
  #Adicionado uma mensagem no Inicio da conversa para mostrar um exemplo de pergunta e resposta que a IA deve obedecer
    user_question = f"""escreva a prova ${questoes_TA[f'Q{x}']}$ usando o sistema de tableaux analitico na linguagem latex na biblioteca qtree"""
    conversa_entrada = [{"role": "user", "content": user_content},
                    {"role": "assistant", "content":assistant_response},
                    {"role": "user", "content":user_question}]
    #Chamada da API
    reposta_ = chamada_api_chatGPT(API_KEY,conversa_entrada)
    try:
      #filtrar apenas a resposta do Modelo
        reposta_ = reposta_["choices"][0]["message"]["content"]
    except:
        print("Um ERRO OCORREU",reposta_ )
    print("Questao:",x)
    print("mensagens de entrada")
    print(conversa_entrada)
    print('resposta')
    print(reposta_)
    #SALVANDO ARQUIVO LATEX
    arquivo_LATEX+= '\n\item '+questoes_TA[f'Q{x}']+' \n\nRESPOSTA CHAT-GPT  \n\n'+reposta_+' \n\n'
    #SALVANDO CSV
    dicionarios_questoes.append({'numero':f'q{x}','questao':questoes_TA[f'Q{x}'],'resposta':reposta_,'resposta_correta':'','avaliacao':''})

Questao: 29
mensagens de entrada
[{'role': 'user', 'content': 'escreva a prova $A\\lor (B\\land C) \\vdash (A\\lor B)\\land (A\\lor C)$ usando o sistema de tableaux analitico na linguagem latex na biblioteca qtree'}, {'role': 'assistant', 'content': '\n\\Tree [.{$T~A\\lor (B\\land C)$ \\\\ $F~(A\\lor B)\\land (A\\lor C)$} [.{$F~A\\lor B$} [.{{\\color{blue}$F~A$} \\\\ {\\color{blue}$F~B$}} [.{\\color{blue}{$T~A$}} [.{$\\times$} ] ] [.{$T~B\\land C$} [.{{\\color{blue}$T~B$} \\\\ $T~C$} [.{$\\times$} ] ] ] ] ] [.{$F~A\\lor C$} [.{{\\color{blue}$F~A$} \\\\ {\\color{blue}$F~C$}} [.{\\color{blue}{$T~A$}} [.{$\\times$} ] ] [.{$T~B\\land C$} [.{$T~B$ \\\\ {\\color{blue}$T~C$}} [.{$\\times$} ] ] ] ] ] ]\n'}, {'role': 'user', 'content': 'escreva a prova $$A\\lor B, \\lnot B \\vdash A$$ usando o sistema de tableaux analitico na linguagem latex na biblioteca qtree'}]
resposta
Para representar a prova \(A \lor B, \lnot B \vdash A\) usando o sistema de tableaux analítico com a biblioteca `qtree` no 

In [8]:
arquivo_LATEX

'\n\\item $\\vdash (A\\lor (A\\land B))\\rightarrow A$ \n\nRESPOSTA CHAT-GPT  \n\nPara provar a validade da fórmula \\((A \\lor (A \\land B)) \\rightarrow A\\) usando o sistema de tableaux analíticos, primeiro começamos negando a fórmula (com o objetivo de encontrar uma contradição). Isso corresponde a \\(\\neg((A \\lor (A \\land B)) \\rightarrow A) \\equiv (A \\lor (A \\land B)) \\land \\neg A\\). A seguir, construímos a árvore:\n\n```latex\n\\documentclass{article}\n\\usepackage{qtree}\n\n\\begin{document}\n\n\\Tree [.{$T~(A \\lor (A \\land B))$ \\\\ $F~A$}\n    [.{$F~A$} \n        [.{$T~A \\lor (A \\land B)$}\n            [.{$T~A$} [.{$\\times$}] ]\n            [.{$T~A \\land B$} \n                [.{\\color{blue}{$T~A$} \\\\ $T~B$} \n                [.{$\\times$}] \n                ]\n            ]\n        ]\n    ] \n]\n\n\\end{document}\n```\n\nExplicação das etapas:\n- Começamos com \\(T~(A \\lor (A \\land B))\\) e \\(F~A\\).\n- Expandimos \\(T~A \\lor (A \\land B)\\), que se di

In [9]:

dicionarios_questoes

[{'numero': 'q1',
  'questao': '$\\vdash (A\\lor (A\\land B))\\rightarrow A$',
  'resposta': 'Para provar a validade da fórmula \\((A \\lor (A \\land B)) \\rightarrow A\\) usando o sistema de tableaux analíticos, primeiro começamos negando a fórmula (com o objetivo de encontrar uma contradição). Isso corresponde a \\(\\neg((A \\lor (A \\land B)) \\rightarrow A) \\equiv (A \\lor (A \\land B)) \\land \\neg A\\). A seguir, construímos a árvore:\n\n```latex\n\\documentclass{article}\n\\usepackage{qtree}\n\n\\begin{document}\n\n\\Tree [.{$T~(A \\lor (A \\land B))$ \\\\ $F~A$}\n    [.{$F~A$} \n        [.{$T~A \\lor (A \\land B)$}\n            [.{$T~A$} [.{$\\times$}] ]\n            [.{$T~A \\land B$} \n                [.{\\color{blue}{$T~A$} \\\\ $T~B$} \n                [.{$\\times$}] \n                ]\n            ]\n        ]\n    ] \n]\n\n\\end{document}\n```\n\nExplicação das etapas:\n- Começamos com \\(T~(A \\lor (A \\land B))\\) e \\(F~A\\).\n- Expandimos \\(T~A \\lor (A \\land B)\\

In [10]:
labels = ['numero','questao','resposta','resposta_correta','avaliacao']

In [11]:
import csv

try:
    with open("csv_dct.csv", "w") as f:
        writer = csv.DictWriter(f, fieldnames=labels)
        writer.writeheader()
        for elem in dicionarios_questoes:
            writer.writerow(elem)
except IOError:
    print("I/O error")

In [12]:
arquivo = open('latex-resultados.txt','w')
arquivo.write(arquivo_LATEX)
arquivo.close()
